In [1]:
import torch
import os
import numpy as np
from tqdm.notebook import tqdm
import random
from torch.utils.data import Dataset, DataLoader
from torch import nn

In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(0)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

In [3]:
root_dir = "../../datasets/waterbird_augmented/dino_v2_large_features"

In [4]:
class WaterbirdDatasetFeatures(Dataset):
    def __init__(self, dataset_path):
        self.dataset = np.load(dataset_path, allow_pickle=True).item()
        
    def __len__(self):
        return len(self.dataset['features'])

    def __getitem__(self, idx):
        feature = self.dataset['features'][idx]
        label = self.dataset['labels'][idx]
        place = self.dataset['places'][idx]

        return feature, label, place

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear = nn.Sequential(nn.Linear(in_features=2048, out_features=1024, bias=False),
                                        nn.BatchNorm1d(num_features=1024),
                                        nn.ReLU(),
                                        # nn.Dropout(0.5, inplace=False),
                                        nn.Linear(in_features=1024,
                                                  out_features=512, bias=False),
                                        nn.BatchNorm1d(num_features=512),
                                        nn.ReLU(),
                                        # nn.Dropout(0.5, inplace=False),
                                        nn.Linear(in_features=512,
                                                  out_features=1, bias=True),
                                        nn.Sigmoid())

    def forward(self, x):
        out = self.linear(x)
        return out

In [6]:
batch_size = 64
test_filename = 'test_data.npy'
test_set = WaterbirdDatasetFeatures(dataset_path=os.path.join(root_dir, test_filename))
test_loader = DataLoader(test_set, batch_size=batch_size,shuffle=False, pin_memory=True)

In [7]:
saving_folder = 'lr_00001_relu'

for sample_size in [100, 200, 400, 800]:
    for j in range(1, 4):
        sample = f'{sample_size}_sample_{j}'
        trained_checkpoint = f'./trained/{saving_folder}/dino_v2_large_augmented_{sample}.pth'
        model = MLP().to(DEVICE)
        model.load_state_dict(torch.load(trained_checkpoint,
                            weights_only=True, map_location=DEVICE))

        with torch.no_grad():
            inference_data = []
            for features, labels, places in tqdm(test_loader):
                features = features.to(DEVICE)
                labels = labels.numpy()
                places = places.numpy()

                out = model(features)
                preds = (out >= 0.5).to(torch.int64).view(-1).numpy()

                for pred, label, place in zip(preds, labels, places):
                    inference_data.append({
                        'img_label': label,
                        'predicted': pred,
                        'place': place
                    })
            np.save(f"./statistics/{saving_folder}/inference_stats/{sample}.npy", inference_data)

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

...

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]